In [1]:
import pickle
import numpy as np
import tensorflow as tf
tf.set_random_seed(777)
#import matplotlib.pyplot as plt
#from random import shuffle

In [2]:
def unpickle(file):
    with open(file, 'rb') as fo:
        data = pickle.load(fo, encoding='bytes')
    return data

In [3]:
#load the CIFAR-10
def load_CIFAR10(pos, n_chunks=1):
    Xtr = []
    Ytr = []
    for i in range(n_chunks):
        train = unpickle(pos + '/data_batch_{0}'.format(i + 1))
        #print(type(train))
        Xtr.extend(train[b'data'])
        Ytr.extend(train[b'labels'])
        test = unpickle(pos + '/test_batch')
        Xte = test[b'data']
        Yte = test[b'labels']
    return np.array(Xtr), np.array(Ytr), np.array(Xte), np.array(Yte)

In [4]:
# expresses the label data in one-hot encoding.
def onehot_encoding (Ytr, Yte):
    Ytr_onehot = np.zeros((Ytr.size, 10))
    Yte_onehot = np.zeros((Yte.size, 10))
    for i in range(Ytr.size):
        Ytr_onehot[i][Ytr[i]] = 1
    for i in range(Yte.size):
        Yte_onehot[i][Yte[i]] = 1
    return Ytr_onehot, Yte_onehot

In [5]:
# load the train and test data
Xtr, Ytr, Xte, Yte = load_CIFAR10('cifar-10-batches-py', 5)

#print(Xtr.shape)
# image data, each data size is 32*32*3
Xtr = Xtr.reshape(50000, 3, 32, 32).transpose(0,2,3,1).astype("float")
Xte= Xte.reshape(10000, 3, 32, 32).transpose(0,2,3,1).astype("float")
#print(Xtr.shape)
# label data of train and test data, label data is represented by one-hot encoding
Ytr_onehot, Yte_onehot = onehot_encoding(Ytr, Yte)

In [ ]:
print(Xtr.shape)
print(Xtr[0:100].shape)

In [ ]:
print(Ytr_onehot.shape)
print(Ytr_onehot[0:100].shape)

In [6]:
class Model:

    def __init__(self, sess, name):
        self.sess = sess
        self.name = name
        self._build_net()

    def _build_net(self):
        with tf.variable_scope(self.name):
            self.training = tf.placeholder(tf.bool)

            # input place holders
            self.X = tf.placeholder(tf.float32, [None, 32, 32, 3])
            self.Y = tf.placeholder(tf.float32, [None, 10])

            # Layer1
            conv1 = tf.layers.conv2d(inputs=self.X, filters=32, kernel_size=[3, 3],padding="SAME", activation=tf.nn.relu)
            norm1=tf.contrib.layers.batch_norm(conv1)
            #dropout1 = tf.layers.dropout(inputs=pool1,rate=Model.prob, training=self.training)

            #Layer2
            conv2 = tf.layers.conv2d(inputs=pool1, filters=64, kernel_size=[3, 3],padding="SAME", activation=tf.nn.relu)
            norm2=tf.contrib.layers.batch_norm(conv2)
            #pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], padding="SAME", strides=1)
            #dropout2 = tf.layers.dropout(inputs=norm2,rate=Model.prob, training=self.training)

            # Layer3
            conv3 = tf.layers.conv2d(inputs=norm2, filters=128, kernel_size=[3, 3], padding="SAME", activation=tf.nn.relu)
            norm3=tf.contrib.layers.batch_norm(conv3)
            #dropout3 = tf.layers.dropout(inputs=norm3,rate=Model.prob, training=self.training)

            # Layer4
            conv4 = tf.layers.conv2d(inputs= norm3, filters=256, kernel_size=[3, 3],padding="SAME", activation=tf.nn.relu)
            norm4=tf.contrib.layers.batch_norm(conv4)
            #dropout4 = tf.layers.dropout(inputs=norm4,rate=Model.prob, training=self.training)
            pool2 = tf.layers.max_pooling2d(inputs=norm4, pool_size=[2, 2],padding="SAME", strides=2)
            dropout5 = tf.layers.dropout(inputs=pool2,rate=0.7, training=self.training)
            
            # Dense Layer with Relu
            flat = tf.reshape(dropout5, [-1, 256 * 16 * 16])
            dense = tf.layers.dense(inputs=flat, units=256, activation=tf.nn.relu)
            
            norm5=tf.contrib.layers.batch_norm(dense)

            # Logits (no activation) Layer: L5 Final FC 256 inputs -> 10 outputs

            self.W = tf.get_variable("W", shape=[256, 10], initializer=tf.contrib.layers.xavier_initializer())
            self.b = tf.Variable(tf.random_normal([10]))
            self.model = tf.matmul(norm5, self.W) + self.b

        # define cost/loss & optimizer
        self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=self.model, labels=self.Y))
        self.optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(self.cost)

        correct_prediction = tf.equal(tf.argmax(self.model, 1), tf.argmax(self.Y, 1))
        self.accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    def predict(self, x_test, training=False):
        return self.sess.run(self.model,feed_dict={self.X: x_test, self.training: training})

    def get_accuracy(self, x_test, y_test, training=False):
        return self.sess.run(self.accuracy,feed_dict={self.X: x_test,self.Y: y_test, self.training: training})

    def train(self, x_data, y_data, training=True):
        return self.sess.run([self.optimizer, self.cost, self.accuracy], feed_dict={
            self.X: x_data, self.Y: y_data, self.training: training})
    
    def test(self, x_data, y_data, training=False):
        return self.sess.run([self.model, self.accuracy], feed_dict={self.X: x_data, self.Y: y_data, self.training:training})

In [7]:
# hyper parameters
learning_rate = 0.002
training_epochs = 50
batch_size = 100

In [8]:
# initialize
sess = tf.Session()

models = []
num_models = 1
for m in range(num_models):
    models.append(Model(sess, "model" + str(m)))

sess.run(tf.global_variables_initializer())

print('Learning Started!')

# train & test my model
for epoch in range(training_epochs):
    avg_cost_list = np.zeros(len(models))
    train_accuracy= np.zeros(len(models))
    total_batch = int(len(Xtr)/ batch_size)
    
    for i in range(total_batch):
        num=i*batch_size
        batch_xs, batch_ys = Xtr[num:num+batch_size], Ytr_onehot[num:num+batch_size]
        # train each model
        for m_idx, m in enumerate(models):
            _, c, a = m.train(batch_xs, batch_ys)
            avg_cost_list[m_idx] += c/total_batch
            train_accuracy[m_idx] += a/total_batch
            
    print('Epoch:','%04d' % (epoch + 1), 'cost =', avg_cost_list, 'train accuracy =',train_accuracy)

print('Learning Finished!')

Learning Started!
Epoch: 0001 cost = [ 1.35271633] train accuracy = [ 0.52562]
Epoch: 0002 cost = [ 0.90021647] train accuracy = [ 0.68186]
Epoch: 0003 cost = [ 0.69779753] train accuracy = [ 0.7543]
Epoch: 0004 cost = [ 0.55193843] train accuracy = [ 0.8077]
Epoch: 0005 cost = [ 0.41887396] train accuracy = [ 0.8528]
Epoch: 0006 cost = [ 0.32174959] train accuracy = [ 0.88626]
Epoch: 0007 cost = [ 0.2405612] train accuracy = [ 0.91432]
Epoch: 0008 cost = [ 0.18384681] train accuracy = [ 0.93656]
Epoch: 0009 cost = [ 0.15021277] train accuracy = [ 0.94798]
Epoch: 0010 cost = [ 0.12628206] train accuracy = [ 0.95724]
Epoch: 0011 cost = [ 0.10688589] train accuracy = [ 0.96362]
Epoch: 0012 cost = [ 0.09556276] train accuracy = [ 0.96736001]
Epoch: 0013 cost = [ 0.09306888] train accuracy = [ 0.96802001]
Epoch: 0014 cost = [ 0.08243235] train accuracy = [ 0.97122001]
Epoch: 0015 cost = [ 0.07116789] train accuracy = [ 0.97540001]
Epoch: 0016 cost = [ 0.0693458] train accuracy = [ 0.976080

In [10]:
#Test model
test_batch_size=500
total_test_batch = int(len(Xte)/ test_batch_size)
test_accuracy = np.zeros(len(models))

for j in range(total_test_batch):
    num=j*test_batch_size
    batch_xt, batch_yt = Xte[num:num+test_batch_size], Yte_onehot[num:num+test_batch_size]
    #test ecah model
    for m_idx, m in enumerate(models):
        _, a = m.test(batch_xt, batch_yt) 
        test_accuracy[m_idx]+= a/total_test_batch
print('Test Accuracy =', test_accuracy)

Test Accuracy = [ 0.7914]
